In [ ]:
# Importación de dependencias y funciones
import findspark
findspark.init("/media/hugo/Datos/Linux/spark/")

import pyspark
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("understanding_sparksession")
    .getOrCreate() )

from __future__ import print_function
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

from operator import add
from operator import sub
sc = spark.sparkContext
# Crear el contexto de Spark Streaming
ssc = StreamingContext(sc, 5)

# Configura el endpoint para localizar el broker de Kafka
# kafkaBrokerIPPort = "172.20.1.21:9092"
kafkaBrokerIPPort = "127.0.0.1:9092"

# Productor simple (Singleton!)
# from kafka import KafkaProducer


In [ ]:
import kafka
class KafkaProducerWrapper(object):
  producer = None
  @staticmethod
  def getProducer(brokerList):
    if KafkaProducerWrapper.producer != None:
      return KafkaProducerWrapper.producer
    else:
      KafkaProducerWrapper.producer = kafka.KafkaProducer(bootstrap_servers=brokerList, key_serializer=str.encode, value_serializer=str.encode)
      return KafkaProducerWrapper.producer

# Envía métricas a Kafka! (salida)
def sendMetrics(itr):
  prod = KafkaProducerWrapper.getProducer([kafkaBrokerIPPort])
  for m in itr:
    prod.send("metrics", key=m[0], value=m[0]+","+str(m[1]))
  prod.flush()

stream = KafkaUtils.createDirectStream(ssc, ["Quatar_GP_2014"], kafkaParams)
kafkaParams = {"metadata.broker.list": kafkaBrokerIPPort}
def parseLine(line):
  s = line.split("\t")
  try:
    return [{"Id": s[0], "Body": (s[6]), "Type": s[9],"Mentions":s[3]}]
  except Exception as err:
      print("Wrong line format (%s): " % line)
      return []
lines = stream.map(lambda data: str(data[1]))


In [ ]:
orders = lines.flatMap(parseLine).map(lambda pos:pos['Mentions'].split(",")).filter(lambda mention:mention!="").reduceByKey(lambda a,b:a+b)
#numPerType = orders.map(lambda o: (o['Mentions'], 1)).filter(lambda o:o['Mentions']!=None).reduceByKey(add)
orders.repartition(1).saveAsTextFiles("data/output", "txt")
orders.pprint()
ssc.start()

In [ ]:
orders = lines.flatMap(parseLine).map(lambda pos:pos['Body'].split(" ")).filter(lambda word:word.startsWith("#")).map(lambda hastag:(hastag,1).updateStateByKey(lambda vals, totalOpt: sum(vals)+totalOpt if totalOpt != None else sum(vals)).sortBy(lambda x:-x[1])transform(lambda x:x)
orders.pprint()

In [ ]:
orders=lines.flatMap(parseLine).map(lambda pos:pos['Type']).countByValueAndWindow(20,10)
orders.pprint()

In [ ]:
ssc.stop()